In [1]:
using LinearAlgebra
using Plots
using SparseArrays
using Arpack #Eigenvalues of sparse arrays with eigs()
using DifferentialEquations
# using DiffEqFlux
using Optim
using ForwardDiff
using DelimitedFiles

In [3]:
#Def parameters
n_cutoff = 8
N = 2 * n_cutoff + 1
NHilbert = N^2
n = sparse(Diagonal(LinRange(-n_cutoff, n_cutoff, N))) #Perhaps implement using StaticArrays
Id = sparse(I, N, N)
exp_iPhi = spdiagm(-1 => ones(N - 1));

#Def Hamiltonian parameters
E_J_GHz = 10.0 #scale of E_J in units of h*GHz, h-bar = 1, h = 2pi
E_C = 1/100  #Charging energies
E_J = 1.0 #Josephson energies
phi_ext = 0.995pi

prefactor = 4 * E_C
Kinetic = 2pi * E_J_GHz * prefactor * (kron(n^2, Id) .+ kron(Id, n^2))

Potential1 = -2pi * E_J_GHz * kron(1 / 2 * (exp_iPhi .+ exp_iPhi'), Id) # -E_J1 cos(phi1)
Potential2 = -2pi * E_J_GHz * kron(Id, 1 / 2 * (exp_iPhi .+ exp_iPhi')) # -E_J2 cos(phi2)
Pot3Mat = kron(exp_iPhi, exp_iPhi')
Pot3Const = -2pi * E_J_GHz * E_J / 2

function get_Pot3(Φ_ext = phi_ext)
    M = exp(im * Φ_ext) * Pot3Mat
    return Pot3Const .* (M + M')
end

function dPot3_dphi(Φ_ext = phi_ext)
    M = exp(im * Φ_ext) * Pot3Mat
    return (im * Pot3Const) .* (M - M')
end

Potential3 =  get_Pot3() #-E_J3 cos(phi1 - phi2 + phi_ext)
Potential12 = E_J * (Potential1 + Potential2)
KinPot12 = Kinetic + Potential12
chargecoupling = kron(n, Id);

In [7]:
# function limit_func(param,pmin,pmax)
#     if pmin < param < pmax
#         return param
#     elseif param <= pmin
#         return pmin
#     else
#         return pmax
#     end
# end

function limit_func(param,pmin,pmax)
    (pmax - pmin) * (1/(1 + exp(-param))) + pmin
end

function inv_limit_func(param,pmin,pmax)
    # -log( (pmax - pmin  + 2e-8) / (param - pmin + 1e-8) - 1)
    -log( (pmax - pmin) / (param - pmin) - 1)
end

function limit_func_abs(param)
    param^2
end

limit_func_abs (generic function with 1 method)

In [171]:
#[Tᵣ, Tₐ, Tₚ, αmin, ϕ, f, A, Φ_ext, λ]
p = [2, 10, 6, 0.7, 0.5418, 0.9763*2.482542369189332, 1.01556/0.2949509890806259, 0.995pi, 0.05289]
p[1] = sqrt(p[1])
p[2] = sqrt(p[2])#inv_limit_func(p[2],0,15)
p[3] = sqrt(p[3])
p[4] = inv_limit_func(p[4],0.5,1)

function alpha(p, t)
    Tᵣ = limit_func_abs(p[1])
    Tₐ = limit_func_abs(p[2])
    Tₚ = limit_func_abs(p[3]) + 2*Tᵣ
    αmin = limit_func(p[4],0.5,1)
    abs_slope = (1 - αmin)/Tₐ
    if t < Tₐ
        return 1 - abs_slope * t
    elseif t > Tₐ + Tₚ
        return αmin + abs_slope * (t - Tₐ - Tₚ)
    else
        return αmin
    end
end

dalphadp(p, t) = ForwardDiff.gradient(p̃ -> alpha(p̃,t), p)

function envelopes!(E, Edot, Tᵣ, Tₐ, Tₚ, t)
    if Tₐ < t < Tₐ + Tᵣ
        arg = pi / 2 * (t - Tₐ) / Tᵣ
        E *= sin(arg)^2
        Edot *= 2*sin(arg)*cos(arg)*pi/(2*Tᵣ)
    elseif Tₐ + Tᵣ <= t <= Tₐ + Tₚ - Tᵣ
        E *= 1
        Edot *= 0
    elseif Tₐ + Tₚ - Tᵣ < t < Tₐ + Tₚ
        arg = pi / 2 * (Tₐ + Tₚ - t) / Tᵣ
        E *= sin(arg)^2
        Edot *= 2*sin(arg)*cos(arg)*(-pi)/(2*Tᵣ)
    else
        E *= 0
        Edot *= 0
    end
    return (E, Edot)
end

function pulse(p, t)
    Tᵣ = limit_func_abs(p[1])
    Tₐ = limit_func_abs(p[2])
    Tₚ = limit_func_abs(p[3]) + 2*Tᵣ
    # αmin = p[4]
    ϕ = p[5]
    f = p[6]
    A = p[7]
    # Φ_ext = limit_func(p[8],0.961pi,1.039pi)
    λ = p[9]

    amp = pi * A / (Tₚ - Tᵣ)
    E = amp
    Edot = copy(amp)
    E, Edot = envelopes!(E, Edot, Tᵣ, Tₐ, Tₚ, t)
    arg = f*(t - Tₐ) + ϕ
    return E*cos(arg) + Edot*λ*sin(arg)
end

dpulsedp(p, t) = ForwardDiff.gradient(p̃ -> pulse(p̃, t), p)

dpulsedp (generic function with 1 method)

In [172]:
Es = eigvals(Matrix(KinPot12 + Potential3))
ψs = eigvecs(Matrix(KinPot12 + Potential3))
ψ0 = ψs[:,1:2]
sizep = length(p);

function f!(du, u, p, t)
    x = u[:,1:2]
    H = KinPot12 + alpha(p,t) .* Potential3 + pulse(p,t) .* chargecoupling

    du[:,1:2] = -im .* (H * x)
end

function b!(du, u, p, t)
    x = u[:,1:2]
    λ = u[:,3:4]
    ps = u[:,5]

    miH = -im .* (KinPot12 + alpha(p,t) .* Potential3 + pulse(p,t) .* chargecoupling)

    du[:,1:2] = miH * x
    du[:,3:4] = miH * λ
    
    du[1:sizep,5] = (- real(im * tr(λ' * chargecoupling * x)) .* dpulsedp(p, t)
                     - real(im * tr(λ' * Potential3 * x)) .* dalphadp(p,t))
end

b! (generic function with 1 method)

In [173]:
function get_T(p)
    Tᵣ = limit_func_abs(p[1])
    Tₐ = limit_func_abs(p[2])
    Tₚ = limit_func_abs(p[3]) + 2*Tᵣ
    2*Tₐ + Tₚ
end
dTdp(p) = ForwardDiff.gradient(p̃ -> get_T(p̃), p)

dTdp (generic function with 1 method)

In [183]:
function wT(L)
    σ = 75e5
    log(L)^4/σ
end
function dwTdL(L)
    σ = 75e5
    4* log(L)^3 /(σ*L)
end

function f(p)
    T = get_T(p)

    forward_prob = ODEProblem(f!, ψs[:,1:2], (0.0, T), p)
    sol = solve(forward_prob, p=p, save_everystep=false, reltol=1e-12, abstol=1e-12)

    UT = abs2.(ψs[:,1:2]'*sol.u[end])
    L = 0.5 * (2 - UT[1,2] - UT[2,1])
    return L + wT(L) * T#L, wT(L) * T
end

function b(p)
    T = get_T(p)

    forward_prob = ODEProblem(f!, ψs[:,1:2], (0.0, T), p)
    sol_f = solve(forward_prob, p=p, save_everystep=false, reltol=1e-8, abstol=1e-8)

    xT = sol_f.u[end][:,1:2]

    UT = abs2.(ψs[:,1:2]'*xT)
    L = 0.5 * (2 - UT[1,2] - UT[2,1])

    λ = (1 + dwTdL(L) * T) .* cat(ψ0[:,2]*ψ0[:,2]'*xT[:,1],ψ0[:,1]*ψ0[:,1]'*xT[:,2],dims=2)

    ps = zeros(289,1) #only 2*length(p) are possibly used
    bu0 = cat(xT,λ,ps,dims=2)#cat(xT,ψT,λ,ϕ,ps,dims=2);

    backward_prob = ODEProblem(b!, bu0, (T, 0.0), p)
    sol_b = solve(backward_prob, p=p, save_everystep=false, reltol=1e-8, abstol=1e-8)
    
    grad = real(sol_b.u[end][1:sizep,5])
    h̃ = -im .* ((KinPot12 + alpha(p, get_T(p)) .* Potential3 + pulse(p,get_T(p)) .* chargecoupling) * xT)

    grad, - λ'*h̃, L
end
    # UT = abs2.(ψ0'*xT)
    # infidelity = 0.5 * (2 - UT[1,2] - UT[2,1])

    # loss[1] = infidelity
    # grad[:] = real(sol_b.u[end][1:sizep,5])

b (generic function with 1 method)

In [175]:
# sol_b, λ, xT = b(p) ;
grad, dLdT, L = b(p) ;

In [182]:
grad

9-element Vector{Float64}:
 -0.6868060220392537
  0.09123105200142848
 -0.7525547481946666
 -0.23498950990022424
  0.00032513374429069496
 -0.04608649511307297
 -0.00019338873144828588
  0.0
 -0.007291418513880498

In [192]:
real(tr(dLdT))*dTdp(p)

9-element Vector{Float64}:
 3.858659226461246e-12
 8.6282243323741e-12
 3.3416969146626496e-12
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [195]:
wT(L).*dTdp(p)

9-element Vector{Float64}:
 0.00894611171208815
 0.02000411392253613
 0.007747560007761834
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [199]:
dp = zeros(Float64,size(p))
dp[2] = 1e-6
res1 = f(p - dp)
res2 = f(p + dp)

0.04767151822682967

In [201]:
(res2 - res1)/(2*dp[2]), 0.09123105200142848 + 0.02000411392253613

(0.11225675201551044, 0.11123516592396461)

In [198]:
(res2 - res1)/(2*dp[1]), -0.6868060220392537 + 0.00894611171208815

(-0.6874293750433121, -0.6778599103271655)

In [61]:
(res2 - res1)/(2*dp[1]), 0.0011102195813677887

(0.0011102906094517806, 0.0011102195813677887)

In [63]:
(res2 - res1)/(2*dp[2]), -0.0001487009444301983

(-0.00014696657779644795, -0.0001487009444301983)

In [65]:
(res2 - res1)/(2*dp[3]), 0.001216524164690993

(0.0012170017771317987, 0.001216524164690993)